# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------
from sklearn.model_selection import  cross_validate ,RepeatedKFold , RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification




### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
#word net installation:

#unmark if you want to use and need to install
#!pip install wn
#!python -m wn download omw-he:1.4

In [4]:
#word net import:

#unmark if you want to use:
#import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
#Hebrew tokenizer installation:

#unmark if you want to use and need to install:
!pip install hebrew_tokenizer

Defaulting to user installation because normal site-packages is not writeable


In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
#import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

# Clean data set

In [10]:
df_train['story'] = df_train['story'].str.replace('\s+', '').str.replace(',', '').str.replace('\d+', '')
df_test['story'] = df_test['story'].str.replace('\s+', '').str.replace(',', '').str.replace('\d+', '')


# אופציונלי(איזון בין גברים לנשים בעמודות המגדר) 

In [11]:
# from sklearn.utils import resample

# # separate the majority and minority classes
# df_majority = df_train[df_train['gender'] == 'm']
# df_minority = df_train[df_train['gender'] == 'f']
# #downsample the majority class
# df_majority_downsampled = resample(df_majority, 
#                                     replace=False,    # sample without replacement
#                                    n_samples=len(df_minority),    # to match minority class
#                                     random_state=42)  # reproducible results

# #combine majority class with minority class
# df_train= pd.concat([df_majority_downsampled, df_minority])


In [12]:
train_corpus=df_train['story'].values
test_corpus=df_test['story'].values


## Vectorization+ Normalization function


In [13]:
def vect_norm_func(vect_method,max_features,norm):##(TfidfVectorizer,1000,'l1')
    count_vect = vect_method(max_features=1000)
    X_train_counts = count_vect.fit_transform(train_corpus)
    #pd.DataFrame(X_train_idf.toarray(), columns=count_vect.get_feature_names_out())
    X_train_counts.shape
    ### normalize the data
    X_train_counts_normalized = preprocessing.normalize(X_train_counts, norm=norm)
    return X_train_counts_normalized

# Calculate function

In [14]:
from sklearn.model_selection import GridSearchCV

def calculate_avg_f11(model, x_train_norm, param_grid):
    # define the evaluation metric
    scorer = make_scorer(f1_score, average='macro')

    # perform grid search for hyperparameter tuning
    clf = GridSearchCV(model, param_grid, scoring=scorer, cv=10)
    clf.fit(x_train_norm, df_train['gender'])

    # print the results for each fold
    for i, params in enumerate(clf.cv_results_['params']):
        # train the model with the best hyperparameters found in this fold
        model.set_params(**params)
        model.fit(x_train_norm, df_train['gender'])

        # calculate the f1 scores for each gender class
        y_pred = model.predict(x_train_norm)
        f1_male = f1_score(df_train['gender'], y_pred, pos_label='m', average='macro')
        f1_female = f1_score(df_train['gender'], y_pred, pos_label='f', average='macro')

        # calculate the average F1 score across all classes
        average_f1 = (f1_male + f1_female) / 2

        # print the results for this fold
        print(f"Fold {i+1}: Best params = {params}, Average F1 score = {average_f1}")


# ## Vectorization+ Normalization


In [15]:
x_train_norm1=vect_norm_func(TfidfVectorizer,1000,'l1')

In [16]:
x_train_norm2=vect_norm_func(TfidfVectorizer,1000,'l2')

In [17]:
x_train_norm3=vect_norm_func(CountVectorizer,1000,'l2')

# Call functions with different hyper para

# MultinomialNB

In [18]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
param_grid = {'alpha': [0.1, 0.2, 0.5]}
calculate_avg_f11(model, x_train_norm1, param_grid)

Fold 1: Best params = {'alpha': 0.1}, Average F1 score = 0.8738552602188965
Fold 2: Best params = {'alpha': 0.2}, Average F1 score = 0.8279532725611043
Fold 3: Best params = {'alpha': 0.5}, Average F1 score = 0.6877404259479651


In [19]:
model = MultinomialNB()
param_grid = {'alpha': [0.6, 0.8, 1.0]}
calculate_avg_f11(model, x_train_norm1, param_grid)

Fold 1: Best params = {'alpha': 0.6}, Average F1 score = 0.6840930005954636
Fold 2: Best params = {'alpha': 0.8}, Average F1 score = 0.4505368497863026
Fold 3: Best params = {'alpha': 1.0}, Average F1 score = 0.444746103569633


In [20]:
model = MultinomialNB()
param_grid = {'alpha': [0.1, 0.2, 0.5]}
calculate_avg_f11(model, x_train_norm2, param_grid)

Fold 1: Best params = {'alpha': 0.1}, Average F1 score = 0.8775120276808976
Fold 2: Best params = {'alpha': 0.2}, Average F1 score = 0.8775120276808976
Fold 3: Best params = {'alpha': 0.5}, Average F1 score = 0.8573077086250054


In [21]:
model = MultinomialNB()
param_grid = {'alpha': [0.1, 0.2, 0.5]}
calculate_avg_f11(model, x_train_norm3, param_grid)

Fold 1: Best params = {'alpha': 0.1}, Average F1 score = 0.8775120276808976
Fold 2: Best params = {'alpha': 0.2}, Average F1 score = 0.8775120276808976
Fold 3: Best params = {'alpha': 0.5}, Average F1 score = 0.8573077086250054


Fold 1: Best params = {'alpha': 0.1}, Average F1 score = 0.8775120276808976

x_train_norm2

# SVC

In [22]:
from sklearn.svm import SVC

model = SVC()
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
calculate_avg_f11(model, x_train_norm1, param_grid)

Fold 1: Best params = {'C': 0.1, 'kernel': 'linear'}, Average F1 score = 0.4329819277108434
Fold 2: Best params = {'C': 0.1, 'kernel': 'poly'}, Average F1 score = 0.6877404259479651
Fold 3: Best params = {'C': 0.1, 'kernel': 'rbf'}, Average F1 score = 0.4329819277108434
Fold 4: Best params = {'C': 0.1, 'kernel': 'sigmoid'}, Average F1 score = 0.4329819277108434
Fold 5: Best params = {'C': 1, 'kernel': 'linear'}, Average F1 score = 0.45626805218950234
Fold 6: Best params = {'C': 1, 'kernel': 'poly'}, Average F1 score = 0.7738479687156512
Fold 7: Best params = {'C': 1, 'kernel': 'rbf'}, Average F1 score = 0.8198271914925042
Fold 8: Best params = {'C': 1, 'kernel': 'sigmoid'}, Average F1 score = 0.4786168132942326
Fold 9: Best params = {'C': 10, 'kernel': 'linear'}, Average F1 score = 0.8786779412418331
Fold 10: Best params = {'C': 10, 'kernel': 'poly'}, Average F1 score = 0.8622854232729052
Fold 11: Best params = {'C': 10, 'kernel': 'rbf'}, Average F1 score = 0.8786779412418331
Fold 12: 

In [23]:
model = SVC()
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
calculate_avg_f11(model, x_train_norm2, param_grid)

Fold 1: Best params = {'C': 0.1, 'kernel': 'linear'}, Average F1 score = 0.4329819277108434
Fold 2: Best params = {'C': 0.1, 'kernel': 'poly'}, Average F1 score = 0.4329819277108434
Fold 3: Best params = {'C': 0.1, 'kernel': 'rbf'}, Average F1 score = 0.4329819277108434
Fold 4: Best params = {'C': 0.1, 'kernel': 'sigmoid'}, Average F1 score = 0.4329819277108434
Fold 5: Best params = {'C': 1, 'kernel': 'linear'}, Average F1 score = 0.822552138564864
Fold 6: Best params = {'C': 1, 'kernel': 'poly'}, Average F1 score = 0.8786779412418331
Fold 7: Best params = {'C': 1, 'kernel': 'rbf'}, Average F1 score = 0.8786779412418331
Fold 8: Best params = {'C': 1, 'kernel': 'sigmoid'}, Average F1 score = 0.5307680324038012
Fold 9: Best params = {'C': 10, 'kernel': 'linear'}, Average F1 score = 0.8786779412418331
Fold 10: Best params = {'C': 10, 'kernel': 'poly'}, Average F1 score = 0.8786779412418331
Fold 11: Best params = {'C': 10, 'kernel': 'rbf'}, Average F1 score = 0.8786779412418331
Fold 12: Be

In [24]:
model = SVC()
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
calculate_avg_f11(model, x_train_norm3, param_grid)

Fold 1: Best params = {'C': 0.1, 'kernel': 'linear'}, Average F1 score = 0.4329819277108434
Fold 2: Best params = {'C': 0.1, 'kernel': 'poly'}, Average F1 score = 0.4329819277108434
Fold 3: Best params = {'C': 0.1, 'kernel': 'rbf'}, Average F1 score = 0.4329819277108434
Fold 4: Best params = {'C': 0.1, 'kernel': 'sigmoid'}, Average F1 score = 0.4329819277108434
Fold 5: Best params = {'C': 1, 'kernel': 'linear'}, Average F1 score = 0.5307680324038012
Fold 6: Best params = {'C': 1, 'kernel': 'poly'}, Average F1 score = 0.8786779412418331
Fold 7: Best params = {'C': 1, 'kernel': 'rbf'}, Average F1 score = 0.8786779412418331
Fold 8: Best params = {'C': 1, 'kernel': 'sigmoid'}, Average F1 score = 0.5307680324038012
Fold 9: Best params = {'C': 10, 'kernel': 'linear'}, Average F1 score = 0.8786779412418331
Fold 10: Best params = {'C': 10, 'kernel': 'poly'}, Average F1 score = 0.8786779412418331
Fold 11: Best params = {'C': 10, 'kernel': 'rbf'}, Average F1 score = 0.8786779412418331
Fold 12: B

In [25]:
model = SVC(kernel='poly')
param_grid = {'C': [0.1, 1.0, 10.0], 'gamma': ['scale', 'auto'], 'degree': [2, 3, 4]}
calculate_avg_f11(model, x_train_norm3, param_grid)

Fold 1: Best params = {'C': 0.1, 'degree': 2, 'gamma': 'scale'}, Average F1 score = 0.4329819277108434
Fold 2: Best params = {'C': 0.1, 'degree': 2, 'gamma': 'auto'}, Average F1 score = 0.4329819277108434
Fold 3: Best params = {'C': 0.1, 'degree': 3, 'gamma': 'scale'}, Average F1 score = 0.4329819277108434
Fold 4: Best params = {'C': 0.1, 'degree': 3, 'gamma': 'auto'}, Average F1 score = 0.4329819277108434
Fold 5: Best params = {'C': 0.1, 'degree': 4, 'gamma': 'scale'}, Average F1 score = 0.4786168132942326
Fold 6: Best params = {'C': 0.1, 'degree': 4, 'gamma': 'auto'}, Average F1 score = 0.4329819277108434
Fold 7: Best params = {'C': 1.0, 'degree': 2, 'gamma': 'scale'}, Average F1 score = 0.8786779412418331
Fold 8: Best params = {'C': 1.0, 'degree': 2, 'gamma': 'auto'}, Average F1 score = 0.4329819277108434
Fold 9: Best params = {'C': 1.0, 'degree': 3, 'gamma': 'scale'}, Average F1 score = 0.8786779412418331
Fold 10: Best params = {'C': 1.0, 'degree': 3, 'gamma': 'auto'}, Average F1 s

Fold 6: Best params = {'C': 1, 'kernel': 'poly'}, Average F1 score = 0.8786779412418331

x_train_norm2


# BernoulliNB

In [26]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
param_grid = {'alpha': [0.1, 1.0, 10.0]}
calculate_avg_f11(bnb, x_train_norm1, param_grid)

Fold 1: Best params = {'alpha': 0.1}, Average F1 score = 0.8775120276808976
Fold 2: Best params = {'alpha': 1.0}, Average F1 score = 0.5980850299009837
Fold 3: Best params = {'alpha': 10.0}, Average F1 score = 0.4329819277108434


In [27]:
bnb = BernoulliNB()
param_grid = {'alpha': [0.1, 1.0, 10.0]}
calculate_avg_f11(bnb, x_train_norm2, param_grid)

Fold 1: Best params = {'alpha': 0.1}, Average F1 score = 0.8775120276808976
Fold 2: Best params = {'alpha': 1.0}, Average F1 score = 0.5980850299009837
Fold 3: Best params = {'alpha': 10.0}, Average F1 score = 0.4329819277108434


In [28]:
bnb = BernoulliNB()
param_grid = {'alpha': [0.1, 1.0, 10.0]}
calculate_avg_f11(bnb, x_train_norm3, param_grid)

Fold 1: Best params = {'alpha': 0.1}, Average F1 score = 0.8775120276808976
Fold 2: Best params = {'alpha': 1.0}, Average F1 score = 0.5980850299009837
Fold 3: Best params = {'alpha': 10.0}, Average F1 score = 0.4329819277108434


Fold 1: Best params = {'alpha': 0.1}, Average F1 score = 0.8775120276808976

x_train_norm1

# KNeighborsClassifier

In [29]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
calculate_avg_f11(knn, x_train_norm1, {})

Fold 1: Best params = {}, Average F1 score = 0.48874908100420633


In [30]:
knn = KNeighborsClassifier()
calculate_avg_f11(knn, x_train_norm3, {})

Fold 1: Best params = {}, Average F1 score = 0.6384885100873882


In [31]:
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': [3, 5, 7]}
calculate_avg_f11(knn, x_train_norm1, param_grid)

Fold 1: Best params = {'n_neighbors': 3}, Average F1 score = 0.6807470879635829
Fold 2: Best params = {'n_neighbors': 5}, Average F1 score = 0.48874908100420633
Fold 3: Best params = {'n_neighbors': 7}, Average F1 score = 0.45626805218950234


In [32]:
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': [3, 5, 7]}
calculate_avg_f11(knn, x_train_norm2, param_grid)

Fold 1: Best params = {'n_neighbors': 3}, Average F1 score = 0.6831421762121099
Fold 2: Best params = {'n_neighbors': 5}, Average F1 score = 0.4329819277108434
Fold 3: Best params = {'n_neighbors': 7}, Average F1 score = 0.4329819277108434


In [33]:
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': [3, 5, 7]}
calculate_avg_f11(knn, x_train_norm3, param_grid)

Fold 1: Best params = {'n_neighbors': 3}, Average F1 score = 0.6815343941959969
Fold 2: Best params = {'n_neighbors': 5}, Average F1 score = 0.6384885100873882
Fold 3: Best params = {'n_neighbors': 7}, Average F1 score = 0.6825278123990215


In [34]:
# KNN with different values of n_neighbors and weights
model = KNeighborsClassifier()
param_grid = {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']}
calculate_avg_f11(model, x_train_norm1, param_grid)

Fold 1: Best params = {'n_neighbors': 3, 'weights': 'uniform'}, Average F1 score = 0.6807470879635829
Fold 2: Best params = {'n_neighbors': 3, 'weights': 'distance'}, Average F1 score = 0.6907482078853047
Fold 3: Best params = {'n_neighbors': 5, 'weights': 'uniform'}, Average F1 score = 0.48874908100420633
Fold 4: Best params = {'n_neighbors': 5, 'weights': 'distance'}, Average F1 score = 0.8810620915032681
Fold 5: Best params = {'n_neighbors': 7, 'weights': 'uniform'}, Average F1 score = 0.45626805218950234
Fold 6: Best params = {'n_neighbors': 7, 'weights': 'distance'}, Average F1 score = 0.8810620915032681
Fold 7: Best params = {'n_neighbors': 9, 'weights': 'uniform'}, Average F1 score = 0.4329819277108434
Fold 8: Best params = {'n_neighbors': 9, 'weights': 'distance'}, Average F1 score = 0.8810620915032681


In [35]:
model = KNeighborsClassifier()
param_grid = {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']}
calculate_avg_f11(model, x_train_norm3, param_grid)

Fold 1: Best params = {'n_neighbors': 3, 'weights': 'uniform'}, Average F1 score = 0.6815343941959969
Fold 2: Best params = {'n_neighbors': 3, 'weights': 'distance'}, Average F1 score = 0.6907482078853047
Fold 3: Best params = {'n_neighbors': 5, 'weights': 'uniform'}, Average F1 score = 0.6384885100873882
Fold 4: Best params = {'n_neighbors': 5, 'weights': 'distance'}, Average F1 score = 0.8810620915032681
Fold 5: Best params = {'n_neighbors': 7, 'weights': 'uniform'}, Average F1 score = 0.6825278123990215
Fold 6: Best params = {'n_neighbors': 7, 'weights': 'distance'}, Average F1 score = 0.8810620915032681
Fold 7: Best params = {'n_neighbors': 9, 'weights': 'uniform'}, Average F1 score = 0.5502756508422664
Fold 8: Best params = {'n_neighbors': 9, 'weights': 'distance'}, Average F1 score = 0.8810620915032681


Fold 4: Best params = {'n_neighbors': 5, 'weights': 'distance'}, Average F1 score = 0.8810620915032681

# LinearSVC

In [36]:
from sklearn.svm import LinearSVC

svm = LinearSVC()
calculate_avg_f11(svm, x_train_norm1, {})

Fold 1: Best params = {}, Average F1 score = 0.822552138564864


In [37]:
svm = LinearSVC()
calculate_avg_f11(svm, x_train_norm2, {})

Fold 1: Best params = {}, Average F1 score = 0.8786779412418331


In [38]:
svm = LinearSVC()
calculate_avg_f11(svm, x_train_norm3, {})

Fold 1: Best params = {}, Average F1 score = 0.8786779412418331


In [39]:
svm = LinearSVC()
param_grid = {'C': [0.1, 1.0, 10.0]}
calculate_avg_f11(svm, x_train_norm1, param_grid)

Fold 1: Best params = {'C': 0.1}, Average F1 score = 0.4329819277108434
Fold 2: Best params = {'C': 1.0}, Average F1 score = 0.822552138564864
Fold 3: Best params = {'C': 10.0}, Average F1 score = 0.8786779412418331


In [40]:
svm = LinearSVC()
param_grid = {'C': [0.1, 1.0, 10.0]}
calculate_avg_f11(svm, x_train_norm2, param_grid)

Fold 1: Best params = {'C': 0.1}, Average F1 score = 0.4329819277108434
Fold 2: Best params = {'C': 1.0}, Average F1 score = 0.8786779412418331
Fold 3: Best params = {'C': 10.0}, Average F1 score = 0.8786779412418331


In [41]:
svm = LinearSVC()
param_grid = {'C': [0.1, 1.0, 10.0]}
calculate_avg_f11(svm, x_train_norm3, param_grid)

Fold 1: Best params = {'C': 0.1}, Average F1 score = 0.4329819277108434
Fold 2: Best params = {'C': 1.0}, Average F1 score = 0.8786779412418331
Fold 3: Best params = {'C': 10.0}, Average F1 score = 0.8786779412418331


Fold 2: Best params = {'C': 1.0}, Average F1 score = 0.8786779412418331
    
x_train_norm2

# DecisionTreeClassifier

In [42]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
calculate_avg_f11(dtc, x_train_norm1, {})

Fold 1: Best params = {}, Average F1 score = 0.8810620915032681


In [43]:
dtc = DecisionTreeClassifier()
calculate_avg_f11(dtc, x_train_norm3, {})

Fold 1: Best params = {}, Average F1 score = 0.8810620915032681


In [44]:
dtc = DecisionTreeClassifier()
param_grid = {'max_depth': [3, 5, 7,9,15,20,50,100]}
calculate_avg_f11(dtc, x_train_norm1, param_grid)

Fold 1: Best params = {'max_depth': 3}, Average F1 score = 0.46755574557653096
Fold 2: Best params = {'max_depth': 5}, Average F1 score = 0.48945856068188426
Fold 3: Best params = {'max_depth': 7}, Average F1 score = 0.510511796042618
Fold 4: Best params = {'max_depth': 9}, Average F1 score = 0.5307680324038012
Fold 5: Best params = {'max_depth': 15}, Average F1 score = 0.5597628014335176
Fold 6: Best params = {'max_depth': 20}, Average F1 score = 0.5827448092735772
Fold 7: Best params = {'max_depth': 50}, Average F1 score = 0.7020578348345652
Fold 8: Best params = {'max_depth': 100}, Average F1 score = 0.8514890539740978


In [45]:
model = DecisionTreeClassifier()
param_grid = {'max_depth': [3, 5, 7,10,20,30], 'min_samples_split': [2, 5, 10]}
calculate_avg_f11(model, x_train_norm1, param_grid)

Fold 1: Best params = {'max_depth': 3, 'min_samples_split': 2}, Average F1 score = 0.46755574557653096
Fold 2: Best params = {'max_depth': 3, 'min_samples_split': 5}, Average F1 score = 0.46755574557653096
Fold 3: Best params = {'max_depth': 3, 'min_samples_split': 10}, Average F1 score = 0.46755574557653096
Fold 4: Best params = {'max_depth': 5, 'min_samples_split': 2}, Average F1 score = 0.48945856068188426
Fold 5: Best params = {'max_depth': 5, 'min_samples_split': 5}, Average F1 score = 0.48945856068188426
Fold 6: Best params = {'max_depth': 5, 'min_samples_split': 10}, Average F1 score = 0.48945856068188426
Fold 7: Best params = {'max_depth': 7, 'min_samples_split': 2}, Average F1 score = 0.510511796042618
Fold 8: Best params = {'max_depth': 7, 'min_samples_split': 5}, Average F1 score = 0.510511796042618
Fold 9: Best params = {'max_depth': 7, 'min_samples_split': 10}, Average F1 score = 0.510511796042618
Fold 10: Best params = {'max_depth': 10, 'min_samples_split': 2}, Average F1

In [46]:
model = DecisionTreeClassifier()
param_grid = {'max_depth': [3, 5, 7,10,20,30], 'min_samples_split': [2, 5, 10]}
calculate_avg_f11(model, x_train_norm3, param_grid)

Fold 1: Best params = {'max_depth': 3, 'min_samples_split': 2}, Average F1 score = 0.46755574557653096
Fold 2: Best params = {'max_depth': 3, 'min_samples_split': 5}, Average F1 score = 0.46755574557653096
Fold 3: Best params = {'max_depth': 3, 'min_samples_split': 10}, Average F1 score = 0.46755574557653096
Fold 4: Best params = {'max_depth': 5, 'min_samples_split': 2}, Average F1 score = 0.48945856068188426
Fold 5: Best params = {'max_depth': 5, 'min_samples_split': 5}, Average F1 score = 0.48945856068188426
Fold 6: Best params = {'max_depth': 5, 'min_samples_split': 10}, Average F1 score = 0.48945856068188426
Fold 7: Best params = {'max_depth': 7, 'min_samples_split': 2}, Average F1 score = 0.510511796042618
Fold 8: Best params = {'max_depth': 7, 'min_samples_split': 5}, Average F1 score = 0.510511796042618
Fold 9: Best params = {'max_depth': 7, 'min_samples_split': 10}, Average F1 score = 0.510511796042618
Fold 10: Best params = {'max_depth': 10, 'min_samples_split': 2}, Average F1

Fold 16: Best params = {'max_depth': 30, 'min_samples_split': 2}, Average F1 score = 0.6258057365942948
    
x_train_norm1

# MultinomialNB

In [47]:
# vectorize the data
count_vect = TfidfVectorizer(max_features=1000)
X_train_counts = count_vect.fit_transform(train_corpus)

# normalize the data
X_train_counts_normalized = preprocessing.normalize(X_train_counts, norm='l1')

# create the Multinomial Naive Bayes classifier
clf = MultinomialNB(alpha=0.1)

# fit the model to the normalized training data (x_train_norm2)
clf.fit(x_train_norm2, df_train['gender'])

# vectorize and normalize the test data
X_test_counts = count_vect.transform(test_corpus)
X_test_counts_normalized = preprocessing.normalize(X_test_counts, norm='l1')

# make predictions on the test data
test_pred = clf.predict(X_test_counts_normalized)

# create a DataFrame to store the results
results = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': test_pred})

# print the results
results.head()

MultinomialNB(alpha=0.1)

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


# SVC

In [49]:
# vectorize the data
count_vect = TfidfVectorizer(max_features=1000)
X_train_counts = count_vect.fit_transform(train_corpus)

# normalize the data (using x_train_norm2)
X_train_counts_normalized = x_train_norm2

# create the SVC with the desired parameters
clf = SVC(C=1, kernel='poly')

# fit the model to the training data
clf.fit(X_train_counts_normalized, df_train['gender'])

# vectorize and normalize the test data
X_test_counts = count_vect.transform(test_corpus)
X_test_counts_normalized = preprocessing.normalize(X_test_counts, norm='l1')

# make predictions on the test data
test_pred = clf.predict(X_test_counts_normalized)

# create a DataFrame to store the results
results1 = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': test_pred})

# print the results
results1.head()

SVC(C=1, kernel='poly')

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


# KNeighborsClassifier

In [50]:
count_vect = TfidfVectorizer(max_features=1000)
X_train_counts = count_vect.fit_transform(train_corpus)

# normalize the data
X_train_counts_normalized = x_train_norm1

# create the KNN classifier
clf = KNeighborsClassifier(n_neighbors=5, weights='distance')

# fit the model to the training data
clf.fit(X_train_counts_normalized, df_train['gender'])

# vectorize and normalize the test data
X_test_counts = count_vect.transform(test_corpus)
X_test_counts_normalized = preprocessing.normalize(X_test_counts, norm='l1')

# make predictions on the test data
test_pred = clf.predict(X_test_counts_normalized)

# create a DataFrame to store the results
results2 = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': test_pred})

# print the results
results2.head()

KNeighborsClassifier(weights='distance')

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


# LinearSVC

In [51]:
# vectorize the data
count_vect = TfidfVectorizer(max_features=1000)
X_train_counts = count_vect.fit_transform(train_corpus)

# Use x_train_norm2 for normalization
X_train_counts_normalized = x_train_norm2

# Create the LinearSVC classifier with the desired parameters
clf = LinearSVC(C=1.0)

# fit the model to the training data
clf.fit(X_train_counts_normalized, df_train['gender'])

# vectorize and normalize the test data
X_test_counts = count_vect.transform(test_corpus)
X_test_counts_normalized = preprocessing.normalize(X_test_counts, norm='l1')

# make predictions on the test data
test_pred = clf.predict(X_test_counts_normalized)

# create a DataFrame to store the results
results3 = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': test_pred})

# print the results
results3.head()

LinearSVC()

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


# DecisionTreeClassifier

In [52]:
# vectorize the data
count_vect = TfidfVectorizer(max_features=1000)
X_train_counts = count_vect.fit_transform(train_corpus)

# normalize the data
X_train_counts_normalized = x_train_norm1

# create the DecisionTreeClassifier with the desired parameters
clf = DecisionTreeClassifier(max_depth=30, min_samples_split=2)

# fit the model to the training data
clf.fit(X_train_counts_normalized, df_train['gender'])

# vectorize and normalize the test data
X_test_counts = count_vect.transform(test_corpus)
X_test_counts_normalized = preprocessing.normalize(X_test_counts, norm='l1')

# make predictions on the test data
test_pred = clf.predict(X_test_counts_normalized)

# create a DataFrame to store the results
results4 = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': test_pred})

# print the results
results4.head()

DecisionTreeClassifier(max_depth=30)

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


# BernoulliNB

In [53]:
# Vectorize the data
count_vect = TfidfVectorizer(max_features=1000, binary=True)
X_train_counts = count_vect.fit_transform(train_corpus)

# Create the Bernoulli Naive Bayes classifier with the best parameter
clf = BernoulliNB(alpha=0.1)

# Fit the model to the normalized training data (x_train_norm1)
clf.fit(x_train_norm1, df_train['gender'])

# Vectorize the test data
X_test_counts = count_vect.transform(test_corpus)

# Make predictions on the test data
test_pred = clf.predict(X_test_counts)

# Create a DataFrame to store the results
results5 = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': test_pred})

# Print the results
results5.head()

BernoulliNB(alpha=0.1)

,test_example_id,predicted_category
0,0,f
1,1,m
2,2,m
3,3,m
4,4,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [54]:
results5.to_csv('classification_results.csv',index=False)